In [7]:
import codecs
import time
import serial
import pandas as pd
import numpy as np
import pyvisa as visa
import datetime
import os
import sys
import monochromatorapi as mcapi
import shutterapi as shapi
import picoapi as pico
import pixisapi as pix
import clr # Import the .NET class library
import sys # Import python sys module
import os # Import os module
from System.IO import * # Import System.IO for saving and opening files
# Import C compatible List and String
from System import String
from System.Collections.Generic import List
# Add needed dll references
sys.path.append(os.environ['LIGHTFIELD_ROOT'])
sys.path.append(os.environ['LIGHTFIELD_ROOT']+"\\AddInViews")
clr.AddReference('PrincetonInstruments.LightFieldViewV5')
clr.AddReference('PrincetonInstruments.LightField.AutomationV5')
clr.AddReference('PrincetonInstruments.LightFieldAddInSupportServices')
# PI imports
from PrincetonInstruments.LightField.Automation import Automation
from PrincetonInstruments.LightField.AddIns import ExperimentSettings
from PrincetonInstruments.LightField.AddIns import CameraSettings
from PrincetonInstruments.LightField.AddIns import DeviceType
from PrincetonInstruments.LightField.AddIns import SensorTemperatureStatus
from PrincetonInstruments.LightField.AddIns import TriggerResponse
import PrincetonInstruments.LightField.AddIns as AddIns

#variables
MCport = 'COM4' #Serial port to monochromator scan controller
shutport = 'COM3' #serial port to shutter controller
wheelport = 'COM1' #port to filter wheel
t = 5.0 #s #wait variable for seconds ot be converted to ms for scanner to stop (useful for goto and scanbasic commands if used)
#gotofromhome function wavelength value
wl = 600.0 #nm 
#gotofrom function wavelength values
wlbegin= 550.0 #nm
wllast= 600.0 #nm
#makescanarray function values
wlstart=550.0 #nm
wlend=600.0 #nm
wlstep=10 # interval nm
exposuretimes=10 #s can be array
#picoammeter function settings
interval = .1 #s time between consecutive writes of selected channel readings to datalog
nsamples = 51  # number of readings total written to datalog
Ch1ON = 1  # 0 = channel off, 1 = channel on
Ch2ON = 0  # 0 = channel off, 1 = channel on
asrl = "asrl5::instr" #instrument address for picoammeter
#PIXIS experiment function settings
MC_lamp = 'Xe'
Traget_spectral_bandpass = 0.62 #nm
Experiment_PIXIS_default='PIXIS_MC_Default' 
exp_filenames_basename='exp_Xe_500ms'
exp_folder_name=exp_filenames_basename+'_180to700nm_10C'
start_wl=180.0 #nm 
end_wl=700.0 #nm 
wl_step=10.0 #nm 
exp_time=1000.0 #ms
n_bias=10 #number of bias frames

#when writing a command, write mcapi.Commandname(variables)
#you must run mcapi.home(port) in between two movement commands
#mcapi.getports()
#mcapi.initial(MCport)
mcapi.home(MCport)
#mcapi.go_to_fromhome(MCport,450)
#mcapi.go_to_from(MCport,270,280)
# mcapi.stop(MCport)
#mcapi.makescanarray(wlstart,wlend,wlstep,exposuretimes)

home circuit enabled, prepared to home
scanner is below home so moving up to home
increasing wavelength at a rate of 23KHz
Limits status is:2
scan controller stopped
decrease wavelength for 3 revolutions
increase wavelength for 2 revolutions
high accuracy circuit enabled
finding edge of home flag at 4500KHz, this will take about 12 seconds
scan controller stopped
homing movement successful
disabled home circuit
homing successful


In [2]:
mcapi.whereishome()

631.26

In [3]:
mcapi.home(MCport)

home circuit enabled, prepared to home
scanner is below home so moving up to home
increasing wavelength at a rate of 23KHz
Limits status is:2
scan controller stopped
decrease wavelength for 3 revolutions
increase wavelength for 2 revolutions
high accuracy circuit enabled
finding edge of home flag at 4500KHz, this will take about 12 seconds
scan controller stopped
homing movement successful
disabled home circuit


In [11]:
start_wl = 225.0
end_wl = 340.0
step_wl = 5.0
wl_list = np.arange(start_wl,end_wl+step_wl,step_wl)
#wl_list=np.array([450,550,650])
#exp_list=np.array([5,10,12])
for idx,wl in enumerate(wl_list):
    if idx==0:
        start=wl_list[idx]
        mcapi.go_to_fromhome(MCport,start)
        #time.sleep(3)
        #time.sleep(exp_list[idx])
        input(f"At {wl}. Press any key go to next wavelength")
    else: 
        from_wl = wl_list[idx-1]
        to_wl=wl_list[idx]
        mcapi.go_to_from(MCport,from_wl,to_wl)
        #time.sleep(3)
        #time.sleep(exp_list[idx])
        input(f"At {wl}. Press any key go to next wavelength")

home circuit enabled, prepared to home
scanner is below home so moving up to home
increasing wavelength at a rate of 23KHz
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:

In [10]:
mcapi.go_to_from(MCport,335,340)

Monochromator is at 335
scan controller is moving for 5 nm
moving status: 0
scan controller stopped


'Movement completed'

In [ ]:
#other variables for rarely used functions in mcapi
#execute user prewritten program stored in non-volitile memory
progname= "C:/Users/User/Downloads/wavelength.csv"
#testing simplest move command of just giving running value
move=2300.0 #step per rev #MUST Run mcapi.stop(port) to stop scanning
Rspeed=2000.0 #steps per second
Startvel=2000.0 #steps per second
Scanvel=2000.0 #steps per second

In [6]:
import pyvisa
import serial
import codecs

string = '4E2105'
enquire = codecs.decode(string,'hex')
rm = pyvisa.ResourceManager()
rm.list_resources()
# ('ASRL1::INSTR', 'ASRL2::INSTR', 'GPIB0::12::INSTR')
inst = rm.open_resource('asrl1::instr')
inst.write_termination='\r'
inst.read_termination='\r'
inst.baud_rate = 9600
inst.data_bits = 8
inst.parity = visa.constants.Parity.none
inst.flow_control = visa.constants.VI_ASRL_FLOW_NONE
inst.write(enquire)
print(inst.query("N!<ENQ>"))
rm.close()

VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

In [6]:
print(inst.query("N!"))

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [38]:
inst.interface_type

InvalidSession: Invalid session handle. The resource might be closed.

In [7]:
import codecs
string = '4E2105'
codecs.decode(string,'hex')

b'N!\x05'

In [7]:
import codecs
import serial

ser = serial.Serial(port='COM1', 
                    baudrate = 9600, 
                    timeout = None,
                    xonxoff = False,
                    parity = serial.PARITY_NONE,
                    stopbits = serial.STOPBITS_ONE,
                    bytesize = serial.EIGHTBITS,
                    )
ser.close()
#string = '4E2105'
#enquire = codecs.decode(string,'hex')
ser.open()
ser.write(b'N!<ENQ>'); #ascii intput 
read = ser.read_until(size=None) #reads out feedback until no data is left
enquire = codecs.decode(read)
ser.close()
msg = f"Program communication initialized"
print(msg)

SerialException: could not open port 'COM1': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

In [ ]:
ser.close()
ser.open()
ser.write(b'N!<ENQ>'); #ascii intput for pressing enter on keyboard
read = ser.read_until(size=None) #reads out feedback until no data is left
enquire = codecs.decode(read)
ser.close()
msg = f"Program communication initialized"
print(msg)